<a href="https://colab.research.google.com/github/cristinarainich/Sternberg_analysis/blob/main/Environmental.%20Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

Updated 16/05 15:45

In [2]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io
from pathlib import Path

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental')

In [5]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental

 D29_Block_1_TrialData.mat	  eeg_envir.csv
 D64_Block_1_TrialData.mat	  eeg_envir_processed.csv
 D65_Block_1_TrialData_1.mat	  Environmental_Subject.mat
 D65_Block_1_TrialData.mat	  healthy
'D71_Block_1_TrialData (1).mat'   ieeg_envir.csv
'D73_Block_1_TrialData (2).mat'   ieeg_envir_processed.csv
'D77_Block_1_TrialData (1).mat'   stimuli


In [6]:
# Patients
ieeg = scipy.io.loadmat(
    working_path.joinpath('Environmental_Subject.mat')
    )

subj1 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData.mat')
    )

subj65 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData_1.mat')
    )

subj64 = scipy.io.loadmat(
    working_path.joinpath('D64_Block_1_TrialData.mat')
)
subj73 = scipy.io.loadmat(
    working_path.joinpath('D73_Block_1_TrialData (2).mat')
    )
subj77 = scipy.io.loadmat(
    working_path.joinpath('D77_Block_1_TrialData (1).mat')
    )

In [7]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/healthy')

In [8]:
# healthy
files = [] 
begin = 'E'
end = '_Block_1_TrialData.mat'
files.append(begin + '32' + end)
for i in range(34, 47):
  files.append(begin + str(i) + end)

In [9]:
files[0]

'E32_Block_1_TrialData.mat'

In [10]:
eeg = []
for i in files:
  subset = scipy.io.loadmat(
      working_path.joinpath(i)
  )
  eeg.append(subset)

#Working with stimuli

In [ ]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/stimuli')

In [ ]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental/stimuli

imaginability.txt	 proposed_nw.mat	 proposed_w3.mat  untrn_w.mat
proposed_nw3_global.mat  proposed_w2.mat	 proposed_w.mat
proposed_nw3.mat	 proposed_w3_global.mat  untrn_nw.mat


In [ ]:
stimuli_words = scipy.io.loadmat(
    working_path.joinpath('proposed_w3.mat')
)

In [ ]:
columns = stimuli_words['proposed_w'].dtype.names
stim_words = pd.DataFrame(columns=columns)
for i in range(0, stimuli_words['proposed_w'].shape[1]):
  values = []
  subset = stimuli_words['proposed_w'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_words.loc[len(stim_words.index)] = values

In [ ]:
stimuli = []
for i in stim_words.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_words.SFreq.values:
  freq.append(i[0][0])

imagine = []
for i in stim_words.imaginability.values:
  imagine.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('word')

In [ ]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
words = pd.DataFrame(columns=columns)
words['Category'] = category
words['Stimulus'] = stimuli
words['Frequency'] = freq
words['Imaginability'] = imagine

In [ ]:
stimuli_nonwords = scipy.io.loadmat(
    working_path.joinpath('proposed_nw3.mat')
)

In [ ]:
columns = stimuli_nonwords['proposed_nw'].dtype.names
stim_nonwords = pd.DataFrame(columns=columns)
for i in range(0, stimuli_nonwords['proposed_nw'].shape[1]):
  values = []
  subset = stimuli_nonwords['proposed_nw'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_nonwords.loc[len(stim_nonwords.index)] = values

In [ ]:
stimuli = []
for i in stim_nonwords.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_nonwords.SFreq.values:
  freq.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('nonword')

In [ ]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
nonwords = pd.DataFrame(columns=columns)
nonwords['Category'] = category
nonwords['Stimulus'] = stimuli
nonwords['Frequency'] = freq
nonwords['Imaginability'] = np.nan

In [ ]:
all_stim = pd.concat([words, nonwords])

# iEEG dataset

In [11]:
data_ieeg = ieeg['Subject']
print('Number of subjects in iEEG dataset is', data_ieeg.shape[1])

Number of subjects in iEEG dataset is 19


In [12]:
subjects_ieeg = [] # subject numbers
for i in data_ieeg['Name'][0]:
  subjects_ieeg.append(i[0])
print(subjects_ieeg)

['D27', 'D28', 'D30', 'D31', 'D32', 'D35', 'D37', 'D38', 'D39', 'D41', 'D42', 'D47', 'D53', 'D54', 'D55', 'D57', 'D63', 'D69', 'D71']


In [13]:
columns = []
for i in range(0, data_ieeg.shape[1]):
  actual = data_ieeg[0][i]['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [14]:
def get_to_df(data):
  df = pd.DataFrame(columns=columns)
  df['Trial'] = np.nan
  df['Subject'] = np.nan

  for i in range(0, data.shape[1]):
    # for each subject
    subset = data[0][i]
    subject = subset['Name'][0]
    print(subset['Task'])
    print(subject)
    for trial in range(0, subset['trialInfo'].shape[1]):
      # for each trial
      sub = subset['trialInfo'][0][trial]
      dtype = sub.dtype.names
      values = []
      for name in columns:
        if name in dtype:
          values.append(sub[name])
        else:
          values.append(np.nan)
      values.append(str(trial)) # adding a trial number
      values.append(subject)
      df.loc[len(df.index)] = values
  return df

In [15]:
ieeg = get_to_df(data_ieeg)

['Environmental_Sternberg']
D27
['Environmental_Sternberg']
D28
['Environmental_Sternberg']
D30
['Environmental_Sternberg']
D31
['Environmental_Sternberg']
D32
['Environmental_Sternberg']
D35
['Environmental_Sternberg']
D37
['Environmental_Sternberg']
D38
['Environmental_Sternberg']
D39
['Environmental_Sternberg']
D41
['Environmental_Sternberg']
D42
['Environmental_Sternberg']
D47
['Environmental_Sternberg']
D53
['Environmental_Sternberg']
D54
['Environmental_Sternberg']
D55
['Environmental_Sternberg']
D57
['Environmental_Sternberg']
D63
['Environmental_Sternberg']
D69
[]
D71


In [16]:
# adding subjects D64
df_64 = pd.DataFrame(columns=columns)
df_64['Trial'] = np.nan
df_64['Subject'] = np.nan
for trial in range(0, subj64['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj64['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D64')
  df_64.loc[len(df_64.index)] = values

In [17]:
# adding subjects D65
df_65 = pd.DataFrame(columns=columns)
df_65['Trial'] = np.nan
df_65['Subject'] = np.nan
for trial in range(0, subj1['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj1['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D65')
  df_65.loc[len(df_65.index)] = values
  i = 96
for trial in range(0, subj65['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj65['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(i)) # adding a trial number
  values.append('D65')
  i += 1
  df_65.loc[len(df_65.index)] = values

In [18]:
ieeg = pd.concat([ieeg, df_64])
ieeg = pd.concat([ieeg, df_65])

In [19]:
# adding subject 73
df_73 = pd.DataFrame(columns=columns)
df_73['Trial'] = np.nan
df_73['Subject'] = np.nan
for trial in range(0, subj73['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj73['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D73')
  df_73.loc[len(df_73.index)] = values

In [20]:
# adding subject 77
df_77 = pd.DataFrame(columns=columns)
df_77['Trial'] = np.nan
df_77['Subject'] = np.nan
for trial in range(0, subj77['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj77['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D77')
  df_77.loc[len(df_77.index)] = values

In [21]:
ieeg = pd.concat([ieeg, df_73])
ieeg = pd.concat([ieeg, df_77])

In [22]:
ieeg.groupby(['Subject'])['Trial'].count()

Subject
D27    160
D28    120
D30    168
D31    168
D32    168
D35    168
D37    168
D38    160
D39    168
D41    168
D42    168
D47    168
D53    168
D54    168
D55    168
D57    168
D63    168
D64    168
D65    168
D69    168
D71    168
D73    168
D77    168
Name: Trial, dtype: int64

In [23]:
# saving ieeg dataset
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental')
ieeg.to_csv(
    Path(working_path).joinpath('ieeg_envir.csv')
)

# Preprocessing of iEEG dataset

In [ ]:
ieeg = pd.read_csv(
    working_path.joinpath('ieeg_envir.csv')
    )

In [24]:
ieeg_alter = pd.read_csv(
    working_path.joinpath('ieeg_envir_processed.csv')
    )

In [25]:
ieeg_alter = ieeg_alter.drop('Unnamed: 0', axis=1)

In [26]:
ieeg.Subject.unique()

array(['D27', 'D28', 'D30', 'D31', 'D32', 'D35', 'D37', 'D38', 'D39',
       'D41', 'D42', 'D47', 'D53', 'D54', 'D55', 'D57', 'D63', 'D69',
       'D71', 'D64', 'D65', 'D73', 'D77'], dtype=object)

In [27]:
ieeg_ = ieeg[['block', 'ProbeType', 'StimlusCategory',
       'ReactionTime', 'Resp', 'RespCorrect', 'Omission',
       'stimulusSounds_idx', 'stimulusSounds_name', 'ProbeCategory', 'probeSound_idx',
       'probeSound_name', 'StimulusCategory', 'Trial',
       'Subject']]

In [28]:
# looks like D27 performed Neighbourhood here
# let's exclude them
ieeg_ = ieeg_[ieeg_['Subject'] != 'D27']

In [29]:
ieeg_ = ieeg_.drop(['Omission', 'ProbeType'], axis=1)

In [30]:
# working on the columns to make them more convinient to use
block = ieeg_.block.values
new_block = []
for i in block:
  while type(i) is np.ndarray:
    i = i[0]
  new_block.append(i)

In [31]:
rt = ieeg_.ReactionTime.values
new_rt = []
for i in rt:
  while type(i) is np.ndarray:
    i = i[0]
  new_rt.append(i)

In [32]:
acc = ieeg_.RespCorrect.values
new_acc = []
for i in acc:
  while type(i) is np.ndarray:
    i = i[0]
  new_acc.append(i)

In [33]:
resp = ieeg_.Resp.values
new_resp = []
for i in resp:
  while type(i) is np.ndarray:
    i = i[0]
  new_resp.append(i)

In [34]:
probe = ieeg_.ProbeCategory.values
new_probe = []
for i in probe:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe.append(i)

In [35]:
probe_idx = ieeg_.probeSound_idx.values
new_probe_idx = []
for i in probe_idx:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_idx.append(i)

In [36]:
stim_idx = ieeg_.stimulusSounds_idx.values
new_stim_idx = []
for i in stim_idx:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0])
  new_stim_idx.append(element)

In [37]:
probe_name = ieeg_.probeSound_name.values
new_probe_name = []
for i in probe_name:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_name.append(i)

In [38]:
stim_name = ieeg_.stimulusSounds_name.values
new_stim_name = []
for i in stim_name:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0][0][0])
  new_stim_name.append(element)

In [44]:
stim1 = ieeg_.StimlusCategory.values
stim2 = ieeg_.StimulusCategory.values
new_stim = []
for i in range(0, ieeg_.shape[0]):
  if type(stim1[i]) is np.ndarray:
    element = stim1[i][0][0][0]
    new_stim.append(element)
  elif type(stim2[i]) is np.ndarray:
    element = stim2[i][0][0][0]
    new_stim.append(element)
  else:
    print(stim1[i], stim2[i])
    break

In [46]:
ieeg_['block'] = new_block
ieeg_['ReactionTime'] = new_rt
ieeg_['RespCorrect'] = new_acc
ieeg_['Resp'] = new_resp
ieeg_['ProbeCategory'] = new_probe
ieeg_['probeSound_idx'] = new_probe_idx
ieeg_['stimulusSounds_idx'] = new_stim_idx
ieeg_['probeSound_name'] = new_probe_name
ieeg_['stimulusSounds_name'] = new_stim_name
ieeg_['StimulusCategory'] = new_stim
ieeg_ = ieeg_.drop('StimlusCategory', axis=1)

In [52]:
ieeg_alter.head()

,block,ReactionTime,Resp,RespCorrect,stimulusSounds_idx,ProbeCategory,probeSound_idx,StimulusCategory,Trial,stimulusSounds_name,probeSound_name,Subject,Length,Words,Nonwords,Environment,ProbeFreq,ProbeImag,StimFreq,StimImag
0,1,1.227901,No,1,"[68, 69, 64, 59, 54]",0,23,nonwords,0,"['thowster', 'viylahs', 'saechahn', 'mahtern',...",yehbahl,D32,5,0,1,0,NaN,NaN,NaN,NaN
1,1,0.953294,Yes,0,"[10, 6, 4, 2, 16, 7, 22, 14, 15]",0,8,environment,1,"['duck_quack', 'crowd_cheering', 'crickets', '...",dog_panting,D32,9,0,0,1,NaN,NaN,NaN,NaN
2,1,1.314829,No,1,"[46, 36, 42, 34, 29]",0,2,words,2,"['tunic', 'madam', 'quota', 'glacier', 'comic']",chalice,D32,5,1,0,0,0.63,4.7,"[0.55, 44.0, 1.86, 0.75, 10.82]","[5.6, 5.6, 3.8, 6.6, 5.4]"
3,1,0.894159,Yes,1,"[36, 33, 28, 27, 46]",1,3,words,3,"['madam', 'duty', 'cherub', 'chapter', 'tunic']",chapter,D32,5,1,0,0,11.84,4.9,"[44.0, 50.96, 0.61, 11.84, 0.55]","[5.6, 4.0, 3.5, 4.9, 5.6]"
4,1,0.866337,No,0,"[71, 51, 55, 61, 59]",1,7,nonwords,4,"['yehbahl', 'drahjer', 'haasahp', 'prehjiy', '...",haasahp,D32,5,0,1,0,NaN,NaN,NaN,NaN


In [51]:
ieeg_.head()

,block,ReactionTime,Resp,RespCorrect,stimulusSounds_idx,stimulusSounds_name,ProbeCategory,probeSound_idx,probeSound_name,StimulusCategory,Trial,Subject
160,1,0.990436,No,1,"[3, 7, 15]","[camera_snapping_photos.wav, cutting_with_scis...",0,14,telephone_dialing.wav,nonwords,0,D28
161,1,1.302180,Yes,1,"[16, 7, 14, 13, 9, 11, 6]","[typing.wav, cutting_with_scissors.wav, teleph...",1,11,hair_dryer.wav,words,1,D28
162,1,1.432630,Yes,1,"[17, 15, 13, 5, 9, 10, 7, 6, 1]","[writing_on_paper.wav, thunder.wav, stream.wav...",1,7,cutting_with_scissors.wav,environment,2,D28
163,1,1.160734,Yes,1,"[17, 1, 13]","[writing_on_paper.wav, bees_buzzing.wav, strea...",1,13,stream.wav,environment,3,D28
164,1,0.845712,Yes,1,"[16, 7, 10, 15, 12, 3, 2]","[typing.wav, cutting_with_scissors.wav, frog_c...",1,3,camera_snapping_photos.wav,environment,4,D28


In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  if len(i) not in length:
    length.append(len(i))
print(length)

In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  length.append(len(i))
eeg_['Length'] = length

In [ ]:
words = []
nonwords = []
environment = []
for i in eeg_.StimulusCategory.values:
  if i == 'nonwords':
    words.append(0)
    nonwords.append(1)
    environment.append(0)
  elif i == 'words':
    words.append(1)
    nonwords.append(0)
    environment.append(0)
  elif i == 'environment':
    words.append(0)
    nonwords.append(0)
    environment.append(1)

# EEG dataset

In [ ]:
subjects = []
for i in files:
  subjects.append(i[:3])

In [ ]:
columns = []
for subject in eeg:
  # for each subject
  actual = subject['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [ ]:
df_ = pd.DataFrame(columns=columns)
df_['Trial'] = np.nan
df_['Subject'] = np.nan
for subject in eeg:
  for trial in range(0, subject['trialInfo'].shape[1]):
    # for each trial
    values = []
    subset = subject['trialInfo'][0][trial]
    dtype = subset.dtype.names
    for name in columns:
      if name in dtype:
        values.append(subset[name])
      else:
        print('check')
        values.append(np.nan)
    values.append(str(trial))
    values.append(subjects[eeg.index(subject)])
    df_.loc[len(df_.index)] = values

In [ ]:
eeg = df_.copy()

In [ ]:
eeg.groupby(['Subject'])['Trial'].count()

Subject
E32    216
E34    216
E35    216
E36    216
E37    216
E38    216
E39    216
E40    184
E41    216
E42    216
E43    216
E44    216
E45    216
E46    216
Name: Trial, dtype: int64

In [ ]:
# saving eeg file 
eeg.to_csv(
    Path(working_path).joinpath('eeg_envir.csv')
)

#Preprocessing of EEG dataset

In [ ]:
eeg = pd.read_csv(
    working_path.joinpath('eeg_envir.csv')
    )

In [ ]:
eeg.Subject.unique()

array(['E32', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41',
       'E42', 'E43', 'E44', 'E45', 'E46'], dtype=object)

In [ ]:
len(eeg.Subject.unique())

14

In [ ]:
eeg_ = eeg[[
    'block', 'StimulusCategory', 'ReactionTime',
       'Resp', 'RespCorrect', 'stimulusSounds_idx',
       'ProbeCategory', 'probeSound_idx',
       'Trial', 'Subject', 'stimulusSounds_name', 'probeSound_name'
]]

Getting rid of additional lists containing lists

In [ ]:
block = eeg_.block.values
new_block = []
for i in block:
  while type(i) is np.ndarray:
    i = i[0]
  new_block.append(i)

In [ ]:
stim_cat = eeg_.StimulusCategory.values
new_stim = []
for i in stim_cat:
  if 'environment' in i:
    new_stim.append('environment')
  elif 'nonwords' in i:
    new_stim.append('nonwords')
  elif 'words' in i:
    new_stim.append('words')

In [ ]:
rt = eeg_.ReactionTime.values
new_rt = []
for i in rt:
  while type(i) is np.ndarray:
    i = i[0]
  new_rt.append(i)

In [ ]:
acc = eeg_.RespCorrect.values
new_acc = []
for i in acc:
  while type(i) is np.ndarray:
    i = i[0]
  new_acc.append(i)

In [ ]:
resp = eeg_.Resp.values
new_resp = []
for i in resp:
  while type(i) is np.ndarray:
    i = i[0]
  new_resp.append(i)

In [ ]:
probe = eeg_.ProbeCategory.values
new_probe = []
for i in probe:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe.append(i)

In [ ]:
probe_idx = eeg_.probeSound_idx.values
new_probe_idx = []
for i in probe_idx:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_idx.append(i)

In [ ]:
stim_idx = eeg_.stimulusSounds_idx.values
new_stim_idx = []
for i in stim_idx:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0])
  new_stim_idx.append(element)

In [ ]:
eeg_['block'] = new_block
eeg_['StimulusCategory'] = new_stim
eeg_['ReactionTime'] = new_rt
eeg_['RespCorrect'] = new_acc
eeg_['Resp'] = new_resp
eeg_['ProbeCategory'] = new_probe
eeg_['probeSound_idx'] = new_probe_idx
eeg_['stimulusSounds_idx'] = new_stim_idx

<ipython-input-58-ea534a877bb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['block'] = new_block
<ipython-input-58-ea534a877bb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['StimulusCategory'] = new_stim
<ipython-input-58-ea534a877bb7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  if len(i) not in length:
    length.append(len(i))
print(length)

[3, 7, 9, 5]


In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  length.append(len(i))
eeg_['Length'] = length

<ipython-input-60-d50296b58efb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Length'] = length


In [ ]:
words = []
nonwords = []
environment = []
for i in eeg_.StimulusCategory.values:
  if i == 'nonwords':
    words.append(0)
    nonwords.append(1)
    environment.append(0)
  elif i == 'words':
    words.append(1)
    nonwords.append(0)
    environment.append(0)
  elif i == 'environment':
    words.append(0)
    nonwords.append(0)
    environment.append(1)

In [ ]:
eeg_['Words'] = words
eeg_['Nonwords'] = nonwords
eeg_['Environment'] = environment

<ipython-input-62-3c31c9b484ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Words'] = words
<ipython-input-62-3c31c9b484ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Nonwords'] = nonwords
<ipython-input-62-3c31c9b484ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [ ]:
probe_names = eeg_.probeSound_name.values
new_probe_names = []
for name in probe_names:
  name = name[0][0][0]
  new_probe_names.append(name)

In [ ]:
stimulus_names = eeg_.stimulusSounds_name.values
new_stim_names = []
for name in stimulus_names:
  name = name[0][0]
  new_name = []
  for j in name:
    new_name.append(j[0][0])
  new_stim_names.append(new_name)

In [ ]:
new_new_stim_names = []
for element in new_stim_names:
  new_element = []
  for name in element:
    new_element.append(name[:-4])
  new_new_stim_names.append(new_element)

In [ ]:
new_new_probe_names = []
for i in new_probe_names:
  new_new_probe_names.append(i[:-4])

In [ ]:
eeg_['stimulusSounds_name'] = new_new_stim_names
eeg_['probeSound_name'] = new_new_probe_names

<ipython-input-67-a747f04adf7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['stimulusSounds_name'] = new_new_stim_names
<ipython-input-67-a747f04adf7e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['probeSound_name'] = new_new_probe_names


**Adding frequency and imaginabilty scores**

In [ ]:
category = eeg_.StimulusCategory.values.tolist()
probe = eeg_.probeSound_name.values
stimuli = eeg_.stimulusSounds_name.values
standard = all_stim[all_stim['Category'] == 'word']['Stimulus'].values.tolist()
standard_freq = all_stim[all_stim['Category'] == 'word']['Frequency'].values.tolist()
standard_imag = all_stim[all_stim['Category'] == 'word']['Imaginability'].values.tolist()

stim_freq = []
stim_imag = []
probe_freq = []
probe_imag = []

for i in range(0, eeg_.shape[0]):
  if category[i] == 'words':
    for name in standard:
      if probe[i] == name:
        freq = standard_freq[standard.index(name)]
        imag = standard_imag[standard.index(name)]
        probe_freq.append(freq)
        probe_imag.append(imag)

    values_freq = []
    values_imag = []      
    for lis in stimuli[i]:
      for name in standard:
        if lis == name:
          freq = standard_freq[standard.index(name)]
          imag = standard_imag[standard.index(name)]
          values_freq.append(freq)
          values_imag.append(imag)
    stim_freq.append(values_freq)
    stim_imag.append(values_imag)
  
  else:
    stim_freq.append(np.nan)
    stim_imag.append(np.nan)
    probe_freq.append(np.nan)
    probe_imag.append(np.nan)

In [ ]:
print(eeg_.shape)
print(len(stim_freq))
print(len(stim_imag))
print(len(probe_freq))
print(len(probe_imag))

(2992, 16)
2992
2992
2992
2992


In [ ]:
eeg_['ProbeFreq'] = probe_freq
eeg_['ProbeImag'] = probe_imag
eeg_['StimFreq'] = stim_freq
eeg_['StimImag'] = stim_imag

<ipython-input-70-c1691c97d5de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeFreq'] = probe_freq
<ipython-input-70-c1691c97d5de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeImag'] = probe_imag
<ipython-input-70-c1691c97d5de>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
# saving the file
eeg_.to_csv(
    Path(working_path).joinpath('eeg_envir_processed.csv')
)